In [1]:
## BASICS
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pickle

## SIGNAL PROCESSING
from scipy.signal import butter, lfilter, filtfilt, welch, stft, iirnotch, freqz
from scipy.fft import fft, fftfreq
from sklearn.decomposition import FastICA
import pywt

## TRADITIONAL ML
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier

## NEURAL NETWORK
from ctgan import CTGANSynthesizer
from ctgan import load_demo


In [2]:
## Loading Data

# Channel Names
n = ['EMG', 'EKG', 'Fp1', 'Fp2', 'A2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4',
     'T4', 'T5', 'P3', 'Pz', 'P4', 'O1', 'O2', 'T1', 'T2', 'PG1', 'PG2', 'A1', 'T6', 'MK']

# Alzheimer's Data
alz_df = []
path = r'/Users/atharvadeshmukh/Desktop/Miniproject_python/alz_csv/'
all_files = glob.glob(os.path.join(path, "*.csv"))
for f in all_files:
    alz_df.append(pd.read_csv(f, names=n))

# Migraine Data
mig_df = []
path = r'/Users/atharvadeshmukh/Desktop/Miniproject_python/mig_csv/'
all_files = glob.glob(os.path.join(path, "*.csv"))
for f in all_files:
    mig_df.append(pd.read_csv(f, names=n))

# Seizure Data
sei_df = []
path = r'/Users/atharvadeshmukh/Desktop/Miniproject_python/seiz_csv/'
all_files = glob.glob(os.path.join(path, "*.csv"))
for f in all_files:
    sei_df.append(pd.read_csv(f, names=n))

# Normal Data
nor_df = []
path = r'/Users/atharvadeshmukh/Desktop/Miniproject_python/nor_csv/'
all_files = glob.glob(os.path.join(path, "*.csv"))
for f in all_files:
    nor_df.append(pd.read_csv(f, names=n))

del all_files
del f
del path


In [3]:
### 6th Order Butterworth filter

def butter_bandpass(lowcut, highcut, fs=128, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs=128, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def bandpass_filter(data):
    out = butter_bandpass_filter(data, 0.5, 60)
    return out

### NOTCH FILTER
    
def notch_filter(data):
    b,a = iirnotch(50, 30, fs=128)
    y = filtfilt(b, a, data)
    return y

## NOISE FILTERING ALL DATA

alz_n = []
for i in range(14):
    x = bandpass_filter(alz_df[i])
    y = notch_filter(x)
    alz_n.append(y)

nor_n = []
for i in range(3):
    x = bandpass_filter(nor_df[i])
    y = notch_filter(x)
    nor_n.append(y)

mig_n = []
for i in range(13):
    x = bandpass_filter(mig_df[i])
    y = notch_filter(x)
    mig_n.append(y)

sei_n = []
for i in range(10):
    x = bandpass_filter(sei_df[i])
    y = notch_filter(x)
    sei_n.append(y)


In [4]:
## SEGMENTATION
def segment_fn(data, fs=128, size=60):
    sgmt_data = []
    sgmt = fs*size
    while(len(data) > sgmt):
        x = data[:sgmt]
        sgmt_data.append(x)
        data = data[sgmt:len(data)]
    return sgmt_data

alz_sg = []
for i in range(14):
    x = segment_fn(alz_n[i])
    alz_sg.append(x)

mig_sg = []
for i in range(13):
    x = segment_fn(mig_n[i])
    mig_sg.append(x)

sei_sg = []
for i in range(10):
    x = segment_fn(sei_n[i])
    sei_sg.append(x)

nor_sg = []
for i in range(3):
    x = segment_fn(nor_n[i])
    nor_sg.append(x)

### TAKING 4 SEGMENTS FROM THE MIDDLE
alz_s = []
nor_s = []
mig_s = []
sei_s = []

for i in range(14):
    if i==8: continue
    alz_sg = segment_fn(alz_n[i])
    alz_sg = alz_sg[7:13]
    final_alz = [*alz_sg[0], *alz_sg[1]]
    final_alz = [*final_alz, *alz_sg[2]]
    final_alz = [*final_alz, *alz_sg[3]]
    final_alz = [*final_alz, *alz_sg[4]]
    final_alz = [*final_alz, *alz_sg[5]]
    #final_alz = alz_sg[0].append(alz_sg[1]).append(alz_sg[2]).append(alz_sg[3])
    alz_s.append(final_alz)

for i in range(3):
    nor_sg = segment_fn(nor_n[i])
    nor_sg = nor_sg[6:13]
    final_nor = [*nor_sg[0], *nor_sg[1]]
    final_nor = [*final_nor, *nor_sg[2]]
    final_nor = [*final_nor, *nor_sg[3]]
    final_nor = [*final_nor, *nor_sg[4]]
    final_nor = [*final_nor, *nor_sg[5]]
    #final_nor = nor_sg[0].append(nor_sg[1]).append(nor_sg[2]).append(nor_sg[3])
    nor_s.append(final_nor)

for i in range(13):
    mig_sg = segment_fn(mig_n[i])
    mig_sg = mig_sg[7:13]
    final_mig = [*mig_sg[0], *mig_sg[1]]
    final_mig = [*final_mig, *mig_sg[2]]
    final_mig = [*final_mig, *mig_sg[3]]
    final_mig = [*final_mig, *mig_sg[4]]
    final_mig = [*final_mig, *mig_sg[5]]
    #final_mig = mig_sg[0].append(mig_sg[1]).append(mig_sg[2]).append(mig_sg[3])
    mig_s.append(final_mig)

for i in range(10):
    sei_sg = segment_fn(sei_n[i])
    sei_sg = sei_sg[7:13]
    final_sei = [*sei_sg[0], *sei_sg[1]]
    final_sei = [*final_sei, *sei_sg[2]]
    final_sei = [*final_sei, *sei_sg[3]]
    final_sei = [*final_sei, *sei_sg[4]]
    final_sei = [*final_sei, *sei_sg[5]]
    #final_sei = sei_sg[0].append(sei_sg[1]).append(sei_sg[2]).append(sei_sg[3])
    sei_s.append(final_sei)


In [5]:
np.shape(alz_s[1])

(46080, 28)

In [13]:
'''
def segment_fn(data, fs=128, size=60):
    sgmt_data = []
    sgmt = fs*size
    while(len(data) > sgmt):
        x = data[:sgmt]
        sgmt_data.append(x)
        data = data[sgmt:len(data)]
    return sgmt_data

alz_s = []
for i in range(14):
    x = segment_fn(alz_n[i])
    alz_s.append(x)

mig_s = []
for i in range(13):
    x = segment_fn(mig_n[i])
    mig_s.append(x)

sei_s = []
for i in range(10):
    x = segment_fn(sei_n[i])
    sei_s.append(x)

nor_s = []
for i in range(3):
    x = segment_fn(nor_n[i])
    nor_s.append(x)

'''

In [5]:
### FEATURES USING DWT cA5 cD5 cD4 cD3 cD2 cD1
def wavelet_fn(data):
    coeffs = pywt.wavedec(data, 'db4', level=5)
    delta, theta, alpha, beta, gamma, noise = coeffs
    wt = pywt.waverec(coeffs, 'db4') ## CHECK WAVEREC VS UPCOEFF FUNCTIONS
    return wt

'''
alz_wt = []
for i in range(15):
    for j in range(len(alz_s[i])):
        x = wavelet_fn(alz_s[i][j])
        alz_wt.append(x)

mig_wt = []
for i in range(13):
    for j in range(len(mig_s[i])):
        x = wavelet_fn(mig_s[i][j])
        mig_wt.append(x)

sei_wt = []
for i in range(10):
    for j in range(len(sei_s[i])):
        x = wavelet_fn(sei_s[i][j])
        sei_wt.append(x)

nor_wt = []
for i in range(3):
    for j in range(len(nor_s[i])):
        x = wavelet_fn(nor_s[i][j])
        nor_wt.append(x)

'''
alz_wt = []
for i in range(13):
    x = wavelet_fn(alz_s[i])
    alz_wt.append(x)

mig_wt = []
for i in range(13):
    x = wavelet_fn(mig_s[i])
    mig_wt.append(x)

sei_wt = []
for i in range(10):
    x = wavelet_fn(sei_s[i])
    sei_wt.append(x)

nor_wt = []
for i in range(3):
    x = wavelet_fn(nor_s[i])
    nor_wt.append(x)



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [7]:
with open('before_gan_wt.pkl', 'wb') as f:
    pickle.dump([alz_wt, nor_wt, mig_wt, sei_wt], f)

In [7]:
final_alz = [*alz_wt[0], *alz_wt[1]]
final_alz = [*final_alz, *alz_wt[2]]
final_alz = [*final_alz, *alz_wt[3]]
final_alz = [*final_alz, *alz_wt[4]]
final_alz = [*final_alz, *alz_wt[5]]
final_alz = [*final_alz, *alz_wt[6]]
final_alz = [*final_alz, *alz_wt[7]]
final_alz = [*final_alz, *alz_wt[8]]
final_alz = [*final_alz, *alz_wt[9]]
final_alz = [*final_alz, *alz_wt[10]]
final_alz = [*final_alz, *alz_wt[11]]
final_alz = [*final_alz, *alz_wt[12]]

np.shape(final_alz)

(599040, 28)

In [8]:
final_nor = [*nor_wt[0], *nor_wt[1]]
final_nor = [*final_nor, *nor_wt[2]]
final_nor = [*final_nor, *mig_wt[1]]
final_nor = [*final_nor, *mig_wt[2]]
final_nor = [*final_nor, *mig_wt[3]]
final_nor = [*final_nor, *mig_wt[4]]
final_nor = [*final_nor, *mig_wt[5]]
final_nor = [*final_nor, *mig_wt[6]]
final_nor = [*final_nor, *sei_wt[1]]
final_nor = [*final_nor, *sei_wt[2]]
final_nor = [*final_nor, *sei_wt[3]]
final_nor = [*final_nor, *sei_wt[4]]


np.shape(final_nor)


(599040, 28)

In [9]:
final_alz = final_alz[:, [0, 1, 2, 3, 4, 5, 6, 13, 14, 23]]
final_nor = final_nor[:, [0, 1, 2, 3, 4, 5, 6, 13, 14, 23]]

TypeError: list indices must be integers or slices, not tuple

In [ ]:
np.shape(final_alz)

In [ ]:
with open('before_gan_wt.pkl', 'wb') as f:
    pickle.dump([final_alz, final_nor], f)

In [6]:
## BANDPASS FILTER

def butter_bandpass(lowcut, highcut, fs=128, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs=128, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def bandpass_filter(data, l, h):
    out = butter_bandpass_filter(data, l, h)
    return out

def sub_bands(data):
    delta = bandpass_filter(data, 0.5, 3.5)
    theta = bandpass_filter(data, 4, 7.5)
    alpha = bandpass_filter(data, 8, 13.5)
    beta = bandpass_filter(data, 14, 36.5)
    gamma = bandpass_filter(data, 37, 49.5)
    return delta, theta, alpha, beta, gamma

In [7]:
'''
## DATASETS FOR SUB-BANDS

alz_sb_beta = []
alz_sb_theta = []
for i in range(13):
    x, y = sub_bands(alz_wt[i])
    alz_sb_beta.append(x)
    alz_sb_theta.append(y)

mig_sb_beta = []
mig_sb_theta = []
for i in range(13):
    x, y = sub_bands(mig_wt[i])
    mig_sb_beta.append(x)
    mig_sb_theta.append(y)

sei_sb_beta = []
sei_sb_theta = []
for i in range(10):
    x, y = sub_bands(sei_wt[i])
    sei_sb_beta.append(x)
    sei_sb_theta.append(y)

nor_sb_beta = []
nor_sb_theta = []
for i in range(3):
    x, y = sub_bands(nor_wt[i])
    nor_sb_beta.append(x)
    nor_sb_theta.append(y)
'''

In [7]:
### STATISTICAL FEATURES 
from scipy.stats import kurtosis, skew
## MEAN
def mean_fn(data):
    x = np.mean(data, axis = 0)
    return x

## VARIANCE
def var_fn(data):
    x = np.var(data, axis = 0)
    return x

## STANDARD DEVIATION
def std_fn(data):
    x = np.std(data, axis = 0)
    return x

## SKEWNESS
def skew_fn(data):
    x = skew(data, axis = 0)
    return x

## KURTOSIS
def kurt_fn(data):
    x = kurtosis(data, axis = 0)
    return x

In [8]:
## LONG SCRIPT TO GET SUB-BANDS AND TO GET THE STATISTICAL FEATURES TOO

alz_final = []
for i in range(13):
    d, t, a, b, g = sub_bands(alz_wt[i])
    m_d = mean_fn(d)
    m_t = mean_fn(t)
    m_a = mean_fn(a) 
    m_b = mean_fn(b) 
    m_g = mean_fn(g) 
    alz_final.append(m_d)
    alz_final.append(m_t)
    alz_final.append(m_a)
    alz_final.append(m_b)
    alz_final.append(m_g)

    v_d = var_fn(d)
    v_t = var_fn(t)
    v_a = var_fn(a) 
    v_b = var_fn(b) 
    v_g = var_fn(g) 
    alz_final.append(v_d)
    alz_final.append(v_t)
    alz_final.append(v_a)
    alz_final.append(v_b)
    alz_final.append(v_g)

    s_d = std_fn(d)
    s_t = std_fn(t)
    s_a = std_fn(a) 
    s_b = std_fn(b) 
    s_g = std_fn(g) 
    alz_final.append(s_d)
    alz_final.append(s_t)
    alz_final.append(s_a)
    alz_final.append(s_b)
    alz_final.append(s_g)

    sk_d = skew_fn(d)
    sk_t = skew_fn(t)
    sk_a = skew_fn(a) 
    sk_b = skew_fn(b) 
    sk_g = skew_fn(g) 
    alz_final.append(sk_d)
    alz_final.append(sk_t)
    alz_final.append(sk_a)
    alz_final.append(sk_b)
    alz_final.append(sk_g)

    k_d = kurt_fn(d)
    k_t = kurt_fn(t)
    k_a = kurt_fn(a) 
    k_b = kurt_fn(b) 
    k_g = kurt_fn(g) 
    alz_final.append(k_d)
    alz_final.append(k_t)
    alz_final.append(k_a)
    alz_final.append(k_b)
    alz_final.append(k_g)

nor_final = []
for i in range(13):
    d, t, a, b, g = sub_bands(mig_wt[i])
    m_d = mean_fn(d)
    m_t = mean_fn(t)
    m_a = mean_fn(a) 
    m_b = mean_fn(b) 
    m_g = mean_fn(g) 
    nor_final.append(m_d)
    nor_final.append(m_t)
    nor_final.append(m_a)
    nor_final.append(m_b)
    nor_final.append(m_g)

    v_d = var_fn(d)
    v_t = var_fn(t)
    v_a = var_fn(a) 
    v_b = var_fn(b) 
    v_g = var_fn(g) 
    nor_final.append(v_d)
    nor_final.append(v_t)
    nor_final.append(v_a)
    nor_final.append(v_b)
    nor_final.append(v_g)

    s_d = std_fn(d)
    s_t = std_fn(t)
    s_a = std_fn(a) 
    s_b = std_fn(b) 
    s_g = std_fn(g) 
    nor_final.append(s_d)
    nor_final.append(s_t)
    nor_final.append(s_a)
    nor_final.append(s_b)
    nor_final.append(s_g)

    sk_d = skew_fn(d)
    sk_t = skew_fn(t)
    sk_a = skew_fn(a) 
    sk_b = skew_fn(b) 
    sk_g = skew_fn(g) 
    nor_final.append(sk_d)
    nor_final.append(sk_t)
    nor_final.append(sk_a)
    nor_final.append(sk_b)
    nor_final.append(sk_g)

    k_d = kurt_fn(d)
    k_t = kurt_fn(t)
    k_a = kurt_fn(a) 
    k_b = kurt_fn(b) 
    k_g = kurt_fn(g) 
    nor_final.append(k_d)
    nor_final.append(k_t)
    nor_final.append(k_a)
    nor_final.append(k_b)
    nor_final.append(k_g)

for i in range(3): 
    d, t, a, b, g = sub_bands(nor_wt[i])
    m_d = mean_fn(d)
    m_t = mean_fn(t)
    m_a = mean_fn(a) 
    m_b = mean_fn(b) 
    m_g = mean_fn(g) 
    nor_final.append(m_d)
    nor_final.append(m_t)
    nor_final.append(m_a)
    nor_final.append(m_b)
    nor_final.append(m_g)

    v_d = var_fn(d)
    v_t = var_fn(t)
    v_a = var_fn(a) 
    v_b = var_fn(b) 
    v_g = var_fn(g) 
    nor_final.append(v_d)
    nor_final.append(v_t)
    nor_final.append(v_a)
    nor_final.append(v_b)
    nor_final.append(v_g)

    s_d = std_fn(d)
    s_t = std_fn(t)
    s_a = std_fn(a) 
    s_b = std_fn(b) 
    s_g = std_fn(g) 
    nor_final.append(s_d)
    nor_final.append(s_t)
    nor_final.append(s_a)
    nor_final.append(s_b)
    nor_final.append(s_g)

    sk_d = skew_fn(d)
    sk_t = skew_fn(t)
    sk_a = skew_fn(a) 
    sk_b = skew_fn(b) 
    sk_g = skew_fn(g) 
    nor_final.append(sk_d)
    nor_final.append(sk_t)
    nor_final.append(sk_a)
    nor_final.append(sk_b)
    nor_final.append(sk_g)

    k_d = kurt_fn(d)
    k_t = kurt_fn(t)
    k_a = kurt_fn(a) 
    k_b = kurt_fn(b) 
    k_g = kurt_fn(g) 
    nor_final.append(k_d)
    nor_final.append(k_t)
    nor_final.append(k_a)
    nor_final.append(k_b)
    nor_final.append(k_g)


for i in range(10):
    d, t, a, b, g = sub_bands(sei_wt[i])
    m_d = mean_fn(d)
    m_t = mean_fn(t)
    m_a = mean_fn(a) 
    m_b = mean_fn(b) 
    m_g = mean_fn(g) 
    nor_final.append(m_d)
    nor_final.append(m_t)
    nor_final.append(m_a)
    nor_final.append(m_b)
    nor_final.append(m_g)

    v_d = var_fn(d)
    v_t = var_fn(t)
    v_a = var_fn(a) 
    v_b = var_fn(b) 
    v_g = var_fn(g) 
    nor_final.append(v_d)
    nor_final.append(v_t)
    nor_final.append(v_a)
    nor_final.append(v_b)
    nor_final.append(v_g)

    s_d = std_fn(d)
    s_t = std_fn(t)
    s_a = std_fn(a) 
    s_b = std_fn(b) 
    s_g = std_fn(g) 
    nor_final.append(s_d)
    nor_final.append(s_t)
    nor_final.append(s_a)
    nor_final.append(s_b)
    nor_final.append(s_g)

    sk_d = skew_fn(d)
    sk_t = skew_fn(t)
    sk_a = skew_fn(a) 
    sk_b = skew_fn(b) 
    sk_g = skew_fn(g) 
    nor_final.append(sk_d)
    nor_final.append(sk_t)
    nor_final.append(sk_a)
    nor_final.append(sk_b)
    nor_final.append(sk_g)

    k_d = kurt_fn(d)
    k_t = kurt_fn(t)
    k_a = kurt_fn(a) 
    k_b = kurt_fn(b) 
    k_g = kurt_fn(g) 
    nor_final.append(k_d)
    nor_final.append(k_t)
    nor_final.append(k_a)
    nor_final.append(k_b)
    nor_final.append(k_g)



In [9]:
with open('before_gan.pkl', 'wb') as f:
    pickle.dump([alz_final, nor_final], f)

In [20]:
'''
## MEAN

alz_mean = []
alz_var = []
alz_std = []
alz_skew = []
alz_kurt = []
for i in range(13):
    a = mean_fn(alz_sb[i])
    b = var_fn(alz_sb[i])
    c = std_fn(alz_sb[i])
    d = skew_fn(alz_sb[i])
    e = kurt_fn(alz_sb[i])
    alz_mean.append(a)
    alz_var.append(b)
    alz_std.append(c)
    alz_skew.append(d)
    alz_kurt.append(e)

mig_mean = []
mig_var = []
mig_std = []
mig_skew = []
mig_kurt = []
for i in range(13):
    a = mean_fn(mig_sb[i])
    b = var_fn(mig_sb[i])
    c = std_fn(mig_sb[i])
    d = skew_fn(mig_sb[i])
    e = kurt_fn(mig_sb[i])
    mig_mean.append(a)
    mig_var.append(b)
    mig_std.append(c)
    mig_skew.append(d)
    mig_kurt.append(e)

sei_mean = []
sei_var = []
sei_std = []
sei_skew = []
sei_kurt = []
for i in range(10):
    a = mean_fn(sei_sb[i])
    b = var_fn(sei_sb[i])
    c = std_fn(sei_sb[i])
    d = skew_fn(sei_sb[i])
    e = kurt_fn(sei_sb[i])
    sei_mean.append(a)
    sei_var.append(b)
    sei_std.append(c)
    sei_skew.append(d)
    sei_kurt.append(e)

nor_mean = []
nor_var = []
nor_std = []
nor_skew = []
nor_kurt = []
for i in range(3):
    a = mean_fn(nor_sb[i])
    b = var_fn(nor_sb[i])
    c = std_fn(nor_sb[i])
    d = skew_fn(nor_sb[i])
    e = kurt_fn(nor_sb[i])
    nor_mean.append(a)
    nor_var.append(b)
    nor_std.append(c)
    nor_skew.append(d)
    nor_kurt.append(e)

'''

In [22]:
## FUNCTION TO APPEND DATA TOGETHER FOR EACH CHANNEL AND GENERATE CLASS LABELS 
## CLASS LABEL '0' FOR ALZHEIMERS AND CLASS LABEL '1' FOR HEALTHY

## CHANGE THIS PART FOR EACH STAT
#w = alz_final        #variance, kurtosis etc
#x = nor_final
#y = mig_mean
#z = sei_mean

#alz_whole = np.vstack([w[0], w[1], w[2], w[3], w[4], w[5], w[6], w[7], w[8], w[9], w[10], w[11], w[12]])
#alz_class = np.array([0] * len(alz_whole))
#alz_class = np.zeros((len(alz_final),1), dtype=int)
#alz_class = pd.DataFrame()
#for i in range(len(alz_whole)):
    #alz_class.append()

#nor_whole = np.vstack([x[0], x[1], x[2]])
#nor_class = np.array([1] * len(nor_whole))
#nor_class = np.ones((len(nor_final),1), dtype=int)

#mig_whole = np.vstack([y[0], y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8], y[9], y[10], y[11], y[12]])
#mig_class = np.ones((len(mig_whole),1), dtype=int)

#sei_whole = np.vstack([z[0], z[1], z[2], z[3], z[4], z[5], z[6], z[7], z[8], z[9]])
#sei_class = np.ones((len(sei_whole),1), dtype=int)

#final_data = np.vstack(alz_whole, nor_whole, mig_whole, sei_whole)
#final_class = np.vstack(alz_class, nor_class, mig_class, sei_class)

alz_final = np.array(alz_final)
nor_final = np.array(nor_final)
### GAN NETWORK TO GENERATE MORE SAMPLES
ctgan = CTGANSynthesizer()
ctgan.fit(alz_final)
alz_test = ctgan.sample(50000)
alz_class = np.zeros((len(alz_test),1), dtype=int)

ctgan = CTGANSynthesizer() #epochs = 10)
ctgan.fit(nor_final)
nor_test = ctgan.sample(50000)
nor_class = np.ones((len(nor_test),1), dtype=int)

### APPENDING DATA TOGETHER
final_whole = np.vstack([alz_test, nor_test]) #, mig_whole, sei_whole])
final_class = np.vstack([alz_class, nor_class]) #, mig_class, sei_class])

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/Library/Frameworks

In [23]:
np.shape(final_class)

(100000, 1)

In [24]:
final_whole = final_whole[:, [0, 1, 2, 3, 4, 5, 6, 13, 14, 23]]
with open('alz_data_100k.pkl', 'wb') as f:
    pickle.dump([final_whole, final_class], f)

In [171]:
## MODEL FUNCTION TO COMPARE CHANNELS
## INPUT THE FINAL DATA AND FINAL CLASS VARIABLES CHANNEL WISE

def svmknn_fn(data, label):
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.20)
    
    ## SVM MODEL
    svm_mdl = SVC(kernel='rbf')
    svm_mdl.fit(X_train, y_train)
    y_pred1 = svm_mdl.predict(X_test)
    #print(confusion_matrix(y_test,y_pred1))
    s = accuracy_score(y_test,y_pred1)

    ## KNN MODEL
    knn_mdl = KNeighborsClassifier(n_neighbors=6)
    knn_mdl.fit(X_train, y_train)
    y_pred2 = knn_mdl.predict(X_test)
    #print(confusion_matrix(y_test,y_pred2))
    k = accuracy_score(y_test,y_pred2)

    return s, k

In [352]:
a = []
b = []

for x in range(10):
    s, k = svmknn_fn(final_whole[ :,[0]], final_class)
    a.append(s)
    b.append(k)

svm_acc = np.mean(a)
kvm_acc = np.mean(b)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.

In [353]:
print("%.4f" %svm_acc)
print("%.4f" %kvm_acc)

0.6974
0.7492


In [11]:
## Taking only the required data to GAN Network. So as determined the channel numbers, 1, 4, 3, 2, 5, 6, 7, 24, 14, 15

## 0,1,2,3,4,5,6,13,14,23

final_whole = final_whole[:, [0, 1, 2, 3, 4, 5, 6, 13, 14, 23]]
with open('alz_data.pkl', 'wb') as f:
    pickle.dump([final_whole, final_class], f)

In [19]:
#final_whole = final_whole[:, [0, 1, 2, 3, 4, 5, 6, 13, 14, 23]]
np.shape(final_class)

(975, 1)

In [24]:
ctgan = CTGANSynthesizer(epochs=10)
ctgan.fit(final_whole)
samples = ctgan.sample(10000)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/Library/Frameworks